In [6]:
import pandas as pd
import numpy as np
from pathlib import Path
import os

# Change to notebook directory to resolve relative paths correctly
notebook_dir = Path(__file__).parent if '__file__' in dir() else Path.cwd()
if 'notebooks' in str(notebook_dir):
    project_root = notebook_dir.parent
else:
    project_root = Path.cwd()
os.chdir(project_root)

# ensure output directory exists
Path("data/cleaned").mkdir(parents=True, exist_ok=True)

# load the provided CAC/LTV model dataset (existing file in data/raw)
df = pd.read_csv("data/raw/cac_ltv_model.csv")

# parse the 'date' column which is in format like 'Jan-23'
df['date'] = pd.to_datetime(df['date'], format='%b-%y', errors='coerce')
df = df.dropna(subset=['date']).copy()

# create month period and a sensible MRR column (use 'arpu' as per-row revenue proxy)
df['month'] = df['date'].dt.to_period('M')
df['MRR'] = df.get('arpu', df.get('plan_price', np.nan))

# aggregate monthly KPIs
monthly = df.groupby('month').agg(
    Revenue=('arpu','sum'),
    Customers=('customer_id','nunique'),
).reset_index()

# compute new customers (first month seen per customer)
first_month = df.groupby('customer_id')['month'].min().reset_index(name='first_month')
new_counts = first_month['first_month'].value_counts().rename_axis('month').reset_index(name='New_Customers')
# align types and merge counts into monthly frame
new_counts['month'] = new_counts['month'].astype(str)
monthly['month'] = monthly['month'].astype(str)
monthly = monthly.merge(new_counts, how='left', on='month')
monthly['New_Customers'] = monthly['New_Customers'].fillna(0).astype(int)

monthly['ARPU'] = monthly['Revenue'] / monthly['Customers']
monthly['Churn_Rate'] = (monthly['Customers'].shift(1) - monthly['Customers']) / monthly['Customers'].shift(1)
monthly['Churn_Rate'] = monthly['Churn_Rate'].fillna(0)

gross_margin = 0.75
monthly['LTV'] = (monthly['ARPU'] * gross_margin) / monthly['Churn_Rate'].replace(0, 0.0001)

monthly.to_csv("data/cleaned/monthly_kpis.csv", index=False)
print('Wrote data/cleaned/monthly_kpis.csv with', len(monthly), 'rows')

Wrote data/cleaned/monthly_kpis.csv with 24 rows
